# Introduction

In [ ]:
# Import libraries and check the versions
import pandas as pd
import sys
import numpy as np
import sklearn
import matplotlib as mpl
import seaborn as sns
import missingno as msno
import xgboost as xgb


print('Python version: {}'.format(sys.version))
print('Numpy version {}'.format(np.__version__))
print('Pandas version {}'.format(pd.__version__))
print('Matplotlib version {}'.format(mpl.__version__))
print('Seaborn version {}'.format(sns.__version__))
print('Sklearn version: {}'.format(sklearn.__version__))
print('Missingno version: {}'.format(msno.__version__))
print("Xgboost version: {}".format(xgb.__version__))

# Pretty display for notebooks
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# for more clear plots
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

Python version: 3.6.3 |Anaconda custom (64-bit)| (default, Oct  6 2017, 12:04:38) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Numpy version 1.14.0
Pandas version 0.20.3
Matplotlib version 2.1.2
Seaborn version 0.8.0
Sklearn version: 0.19.1
Missingno version: 0.3.5
Xgboost version: 0.7


### 1. Data Collection

This dataset can be found at kaggle's website. First column of the dataset is the index column and we specify that with index_col = 0. Let's see the first five records of the dataset.

In [ ]:
# retrieve the data
df = pd.read_csv('h1b_kaggle.csv', index_col=[0])
df.head()

### 2. Data Wrangling

Before we do explonatary data analysis, we need to select necessary features and clean the data. 

In [ ]:
# select the features that will be used creating the model
data = df[['CASE_STATUS', 'SOC_NAME',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'WORKSITE']]

Missigno is a library that allows us to visualize missing data in the dataset.

In [ ]:
# missing values
msno.matrix(data.sample(1000))

In [ ]:
msno.dendrogram(data)

In [ ]:
#check the missing data
data.isnull().sum()

In [ ]:
# remove the missing values
data = data.dropna()

In [ ]:
# convert all strings to uppercase 
data['SOC_NAME'] = data['SOC_NAME'].str.upper()

In [ ]:
# remove everthing after comma from job title
data['SOC_NAME'] = data['SOC_NAME'].apply(lambda x: x.split(', ')[0])

In [ ]:
# There 
data[data['SOC_NAME'].str.contains('CARPI')]

In [ ]:
data = data[~data['SOC_NAME'].str.contains('CARPI')]

Current format of the worksite column is **City Name, State**, for this study we will focus on only state. 

In [ ]:
# remove city names from worksite column
data['WORKSITE'] = data['WORKSITE'].apply(lambda x: x.split(', ')[1])

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

data['PREVAILING_WAGE'].describe()

Clearly, there are outliers in the dataset.

In [ ]:
data[(data['PREVAILING_WAGE'] > 500000) | (data['PREVAILING_WAGE'] < 25000)].shape

 Approxiametly, 12000 wages are below 25000 or above 500000 dollars, those records will be removed.

In [ ]:
cleaned_data = data[(data['PREVAILING_WAGE'] < 500000)]
cleaned_data = cleaned_data[(cleaned_data['PREVAILING_WAGE'] > 25000)]

### 3. Data Exploring

**CASE_STATUS** : This is our target feature. There were 7 possible values in the dataset and we reduced it to 2. Because only one status has a positive result and rest of the statues have a negative result. 

**SOC_NAME** : Type of the job. There are 1584 unique jobs in the dataset.

**FULL_TIME_POSITION** : This column indicates if the job is full time or not. 

**WORKSITE** : Location of the job. Original column had the state and city information. I removed the cities. The model is going to make predictions based on the state information.

In [ ]:
# type of columns
cleaned_data.dtypes

In [ ]:
print ('Number of records: ', cleaned_data.shape[0])
print ('Number of positive cases: ', cleaned_data['CASE_STATUS'].value_counts()[0])
print ('Number of negative cases: ', cleaned_data['CASE_STATUS'].value_counts()[1])
print ('Percentage of positive cases: ', \
       cleaned_data['CASE_STATUS'].value_counts()[0] * 100 / cleaned_data.shape[0])

After removing the null values, we still have close to 3 million records. There are 4 features which are SOC_NAME, FULL_TIME_POSITION, PREVAILING_WAGE and WORKSITE. Our target value is CASE_STATUS. 

In [ ]:
cleaned_data['CASE_STATUS'].value_counts().plot(kind='bar', alpha=0.5)
plt.title('Distribution of case statuses')
plt.ylabel('Frequency')
plt.savefig('Distribution_of_case_status.png');

We have more positive case results than negative results. 

In [ ]:
# number of unique values in each column
for column in cleaned_data:
    print(column, cleaned_data[column].nunique())   

In [ ]:
cleaned_data['WORKSITE'].groupby(cleaned_data['WORKSITE']).count()\
                .sort_values(ascending=False).head(10).plot(kind='bar', alpha=0.5)
plt.title('Top 10 cities for H1-B visa')
plt.savefig('Top_cities.png');

In [ ]:
cleaned_data['FULL_TIME_POSITION'].value_counts().plot(kind='bar', alpha=0.5)
plt.title('Distribution of Full Time - Part Time')
plt.ylabel('Frequency');

In [ ]:
cleaned_data.groupby(['CASE_STATUS','FULL_TIME_POSITION']).count()['SOC_NAME'].\
            unstack().plot(kind='barh',figsize=(12,5), alpha=0.5)
plt.title('Case Status versus Type of position')
plt.ylabel('Frequency');

In [ ]:
i = 'PREVAILING_WAGE'

plt.figure(figsize=(10,8))
plt.subplot(211)
plt.xlim(cleaned_data[i].min(), cleaned_data[i].max()*1.1)

ax = cleaned_data[i].plot(kind='kde')

plt.subplot(212)
plt.xlim(cleaned_data[i].min(), cleaned_data[i].max()*1.1)
sns.boxplot(x=cleaned_data[i]);

Here we have two plots, the density plot and the box plot. This is a good way to view the data as we can see in the density plot (top) that there is some data points in the tails but it is difficult to see, however it is clear in the box plot.

In [ ]:
--

### 4. Data Transforming

For highly sckewed features, it is always good to do transformation. **PREVAILING_WAGE** column has tail on the right and we will apply logarithmic transformation on it. 

In [ ]:
# log transform the data
cleaned_data['Log_' + i] = np.log(cleaned_data[i])

In [ ]:
i = 'Log_PREVAILING_WAGE'

plt.figure(figsize=(10,8))
plt.subplot(211)
plt.xlim(cleaned_data[i].min(), cleaned_data[i].max()*1.1)

ax = cleaned_data[i].plot(kind='kde')

plt.subplot(212)
plt.xlim(cleaned_data[i].min(), cleaned_data[i].max()*1.1)
sns.boxplot(x=cleaned_data[i]);

time to scale transformed data 

In [ ]:
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['Log_PREVAILING_WAGE']

transformed_data = pd.DataFrame(data = cleaned_data)
transformed_data[numerical] = scaler.fit_transform(cleaned_data[numerical])

In [ ]:
# remove original wage column 
del transformed_data['PREVAILING_WAGE']

In [ ]:
# 
transformed_data.head()

4.1 Data Processing

In [ ]:
transformed_data['CASE_STATUS'].unique()

There are seven types of case statues but only the "CERTIFIED" have a positive result. 

In [ ]:
transformed_data['CASE_STATUS'] = transformed_data['CASE_STATUS'].apply(lambda x: 1 if x == 'CERTIFIED' else 0)

In [ ]:
# One-hot encode the transformed data using pandas.get_dummies()
features_final = pd.get_dummies(transformed_data, columns=['SOC_NAME', 'FULL_TIME_POSITION', 'WORKSITE'])

# 
transformed_data['FULL_TIME_POSITION'] = transformed_data['FULL_TIME_POSITION'].apply(lambda x: 1 if x == 'Y' else 0)

# Print the number of features after one-hot encoding
encoded = list(features_final.columns)
print ("total features after one-hot encoding: ", len(encoded))

# Uncomment the following line to see the encoded feature names
#print encoded

In [ ]:
print ("Shape of final features: ", (features_final.shape))

#first 5 rows
features_final.head()

4.2 Train-Test Split

In [ ]:
features_final = features_final[:500000]

In [ ]:
X = features_final.iloc[:,1:]
y = features_final['CASE_STATUS']

In [ ]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print ("Training set has samples: ", (X_train.shape[0]))
print ("Testing set has samples: ",  (X_test.shape[0]))

### 5. Data Modeling

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_1 = LogisticRegression(random_state = 0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_2 = RandomForestClassifier(random_state = 0)

In [ ]:
from xgboost import XGBClassifier

clf_3 = XGBClassifier(random_state = 0)

In [ ]:
clf_1.fit(X_train, y_train)

In [ ]:
clf_2.fit(X_train, y_train)

In [ ]:
clf_3.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

#make predictions for test data
y_pred = clf_1.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#make predictions for test data
y_pred = clf_2.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:

#make predictions for test data
y_pred = clf_3.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

### 6. Model Evaluation

Naive predictor

In [ ]:
# TODO: Calculate accuracy, precision and recall
TP = np.sum(y)
TN = 0
FP = y.count() - np.sum(y)
FN = 0

accuracy = TP / (TP + FP)
recall = TP / (TP + FN)
precision = TP / (TP + FP)

# TODO: Calculate F-score using the formula above for beta = 0.5 and correct values for precision and recall.
# HINT: The formula above can be written as (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
beta = 0.5
fscore = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)

# Print the results 
print ("Naive Predictor: [Accuracy score: ", accuracy, "F-score:" ,fscore)